`sklearn tf-idf`

`jieba依各罪判決書 形成 各罪語料庫`

In [ ]:
crime = ['傷害直系血親尊親屬致死', 
         '傷害直系血親尊親屬', 
         '傷害致死', 
         '過失傷害致重傷', 
         '傷害致重傷', 
         '重傷害未遂', 
         '重傷致死', 
         '重傷害',
         '教唆傷害',
         '肇事逃逸',
         '過失傷害',
         '傷害',
         '無罪']
startTime = time.time()


# jieba
# load 自訂義 dict
dict = 'C:/Users/Jeh/word2vec/dict_judge_update_new.txt'
jieba.load_userdict(dict)
           
# load 自訂義 stopword
stopword = 'C:/Users/Jeh/word2vec/stopword_judge_update.txt'
stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]


for i in range(0, len(crime)):
    
    # step 1 讀取 該罪對應內文
    df = pd.read_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.csv', encoding='utf-8')
    df_content = df.loc[:, 'judge_main_content']

    # step 2 將 內文+正規+結巴 存成 txt
    with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.txt', 'w', encoding='utf-8') as f:
        contlst = []
        for content in df_content:
            contlst.append(content)
        
        data_content = ''.join(contlst)
        #print(data_content)
    
        # re
        contlst_re = []
        r4 = "\【.*?】+|\《.*?》+|\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\，。=？、：“”‘’￥……()《》【】]|[a-zA-Z]+|（|）|，|。|；|○|「|」"
        data_content_ = re.sub(r4,'',data_content)
        contlst_tmp = list(data_content_)
    
        for content in contlst_tmp:
            contlst_re.append(content)
    
        data_content_re = ''.join(contlst_re)
        #print(data_content_re)
        
        data_content_re_lst = str(data_content_re)
        f.write(data_content_re_lst)
    
    with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+11}_{crime[i]}_jeba.txt', 'w', encoding='utf-8') as new_f:
        with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+11}_{crime[i]}.txt', 'r', encoding='utf-8') as f:            
            for times, data in enumerate(f, 1):
                print(f'開始存取 {crime[i]} 的 jieba.txt')
                data = jieba.cut(data)
                data = [word for word in data if word != ' ']
                data = [word for word in data if word not in stopwords]  # 停用字
                data = ' '.join(data)  # 將 list 轉成 string
                new_f.write(data)

print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")

`載入各罪的判決書語料庫`

In [15]:
import pandas as pd
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer



crime = ['1_傷害直系血親尊親屬致死', 
         '2_傷害直系血親尊親屬', 
         '3_傷害致死', 
         '4_過失傷害致重傷', 
         '5_傷害致重傷', 
         '6_重傷害未遂',  
         '7_重傷致死', 
         '8_重傷害',
         '11_過失傷害',
         '12_傷害',  
         '13_無罪',
         '14_其他']
crimeName = crime[2]
doc_new_lst = []

for filename in glob.glob(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}/jieba/*.txt'):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='utf-8') as f:
        txtn = f.read()
#     print(txtn)
        doc_new_lst.append(txtn)

# print(doc_new_lst)
doc = CountVectorizer()
word_count = doc.fit_transform(doc_new_lst)

# 查看判決書的詞頻形狀
word_count.shape

(53, 7860)

`初始化 IDF`

In [16]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=doc.get_feature_names_out(),columns=["idf_weights"])
df_idf.sort_values(by = ['idf_weights']).head(50)

,idf_weights
中華民國,1.000000
前段,1.000000
過失,1.000000
檢察官,1.000000
被告,1.000000
刑事,1.000000
所示,1.000000
之刑,1.000000
訴訟法,1.000000
刑法,1.000000


`TF-IDF計算`

In [18]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(doc_new_lst)
first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[1]

df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index = tfidf_vectorizer.get_feature_names_out(), columns = ["tfidf"])
df.sort_values(by = ["tfidf"],ascending = False).head(45)

,tfidf
以下,0.327034
被告,0.228551
修正,0.197863
拘役,0.182303
有期徒刑,0.181550
108,0.158369
代行,0.148049
刑法,0.146926
宣告,0.144731
萬元,0.138504
